# RNN Based molucule generation 

Laurent Cetinsoy

In this hands-on we want to generate molecule formulas for denovo-drug discovery. 

For that we need to use Generative models. Generative models are models which goes beyond classification or simple regression : they are able to generate data that look like previously seens dataset. 

There exists a lot of models : 

- Bayesian models like graphical models
- Recurrent models (for sequence generation like texte)
- Variational auto encoders
- Generative adversarial models
- Flow and diffusion models 


In the hands-on we will start by  trainning a character based RNN to generate smile molecules


We want to feed smile representations of molecules to an RNN.
The basic idea is we will train it to predict the next smile token of a molecule given the previous one. 

For instance for the following molecule "CC(=O)NC1=CC=C(O)C=C1" will may give to the model

X = "CC(=O)N" 
y = C

and ask the RNN to learn to predict y given X

Like a standard language model !


## RNN Language model 


A language model is a model which predict the next token of a sequence given the previous ones : 

$ P(X_t | X_{t-1}, X_{t-2}, ..., X_{t-p})  $


This model can be learned with a Recurrent neural network 

$ y = P(X_t | X_{t-1}, X_{t-2}, ..., X_{t-p}) = RNN_{\theta} (X_{t-1}, X_{t-2}, ..., X_{t-p})  $


In order to train such model you need a corpus of data. 



There are two main ways to do that : Word level model or character level model

For character level models, an interesting resource is : http://karpathy.github.io/2015/05/21/rnn-effectiveness/



Explain briefly what is the difference between word based language model and character based language model 

## Loading the data

Dowload the following dataset : https://github.com/joeymach/Leveraging-VAE-to-generate-molecules

In [ ]:
!git clone https://github.com/joeymach/Leveraging-VAE-to-generate-molecules.git ./data/

Cloning into './data'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 40 (delta 16), reused 5 (delta 2), pack-reused 0
Receiving objects: 100% (40/40), 8.46 MiB | 5.09 MiB/s, done.
Resolving deltas: 100% (16/16), done.


Import pandas and load the first 1000 lines

In [ ]:
import pandas as pd

# Import the first 1000 lines
df = pd.read_csv('./data/250k_smiles.csv', nrows=1000)

Display the first rows of the dataframe

In [ ]:
df.head(1)

,smiles,logP,qed,SAS
0,CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1\n,5.0506,0.702012,2.084095


## Processing the data

We need to do the following things : 

- convert smile tokens to numbers
- build  smile token sequences and corresponding labels pairs

Compute the biggest smile molecule size

In [ ]:
df['smiles'].str.len().max()

106


Code a function **unic_characters(string)** which return the unic characters in a string


In [ ]:
def unic_characters(string):
    rtn = set()
    
    for char in string:
        rtn.add(char)
        
    return rtn

Concatenate all smile string of the pandas dataframe and use **unic_characters** to get the unic_characters 

In [ ]:
unic_chars = unic_characters(" ".join(df['smiles']))
nb_chars = len(unic_chars) + 1
len(unic_chars)

34

Code a function **map_char_to_int(unic_chars)** which returns a dictionnary where each char is assigned an int value. 
Add a character to specify the end of the molecule (like "\n")


In [ ]:
ending_tocken = "\n"

def map_char_to_int(unic_chars):
    rtn = dict()
    
    for i, char in enumerate(unic_chars):
        rtn[char] = i
        
    rtn[ending_tocken] = len(unic_chars)
    return rtn

Code a function map_int_to_char(unic_chars) which returns the reverse mapping. 

If you want you can merge both functions in a class

In [ ]:
def map_int_to_char(unic_chars):
    rtn = dict()
    
    for i, char in enumerate(unic_chars):
        rtn[i] = char
        len(unic_chars)
        
    rtn[len(unic_chars)] = ending_tocken
    return rtn

For each smile molecule add the ending token to it

In [ ]:
df["smiles"] = df["smiles"].apply(lambda x: x + ending_tocken)

In [ ]:
dict_char_to_int = map_char_to_int(unic_chars)
dict_int_to_char = map_int_to_char(unic_chars)

## Building the dataset

Now we will create the dataset so that it has the good share for our Keras LSTM model

Remember Keras recurrent models expect a 3D array with shapes (n_examples, seq_len, n_features)



What will be n_features in our case ? 

Ca sera 3 car on a 3 autres colonnes

Code a function **build_X_and_y(string, i_char, seq_lenght)** which takes a string, a **seq_length** number and a position. 


It should create X by by getting all character between i and i + seq_length 
and create y by getting the character following the X sequence
it returns X and y

Test your function on the following string "" with seq_length = 4 and i = [1, 2, 3]

In [ ]:
(n_examples, seq_len, n_features) = (len(df),5,1)

def build_X_and_y(string, i_char, seq_length):
    if(i_char >= len(string) or i_char + seq_length >= len(string)):
        return "", ""
    
    X = string[i_char:i_char + seq_length]
    y = string[i_char + seq_length]
    return X, y

for i in range(1, 4):
    print(build_X_and_y("azertyuiop", i, 4))

('zert', 'y')
('erty', 'u')
('rtyu', 'i')


In [ ]:
build_X_and_y("azertyuiop", 0, seq_len)

('azert', 'y')

By using build_X_and_y and map_char_to_int build a list nameed X_train and a list named y_train 

In [ ]:
X_train = []
y_train = []

for i in df['smiles'].tolist():
    
    X, y = build_X_and_y(i, 0, seq_len)
    
    X_ = []

    y_ = [0] * nb_chars
    y_[dict_char_to_int[y]] = 1
    
    for e in X:
        X_.append(dict_char_to_int[e])
    
    X_train.append(X_)
    y_train.append(y_)
    
X_train[0]

[23, 23, 17, 23, 4]

Create numpy arrays from the lists

In [ ]:
import numpy as np
X_train = np.array(X_train)
y_train = np.array(y_train)

X_train.shape

(1000, 5)

Reshape the X numpy array (n_examples, seq_lenght, 1)

In [ ]:
seq_length = seq_len
X_train = X_train.reshape(n_examples, seq_length, 1)

Normalize X by dividing each values by the total number of unic characters

In [ ]:
X_train = X_train /len(unic_chars)

Import Keras and build (at least) a two layered LSTM network with 128 neurone in each.

You can also add Dropoutlayers

Do you think you should use the return_sequences = True ? If yes, when ? 


Add a Dense layer on top with with the appropriate activation function and number of neurones 


In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense

lstm_neurons = 128

dropout_rate = 0.2

dense_neurons = 36

dense_activation = 'softmax'

model = Sequential()

model.add(LSTM(lstm_neurons, input_shape=(seq_length, 1), return_sequences=True))
model.add(Dropout(dropout_rate))
model.add(LSTM(lstm_neurons))
model.add(Dropout(dropout_rate))

# Add the Dense layer
model.add(Dense(nb_chars, activation=dense_activation))

2023-01-19 19:35:50.455574: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-19 19:35:50.593270: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-19 19:35:50.593303: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-19 19:35:51.396817: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

Compile the model with the appropriate loss function and the adam optimizer

In [ ]:
from keras.optimizers import Adam

loss_function = 'categorical_crossentropy'

optimizer = Adam()

model.compile(loss=loss_function, optimizer=optimizer, metrics=['accuracy'])



Train the model on 20 epochs and 10 examples (yeah you read correctly) and check that the model overfits ! 

In [ ]:
model.fit(X_train[:10], y_train[:10], epochs=20, batch_size=1)

Epoch 1/20
10/10 [==============================] - 2s 4ms/step - loss: 3.5392 - accuracy: 0.1000 
Epoch 2/20
10/10 [==============================] - 0s 5ms/step - loss: 3.3551 - accuracy: 0.3000
Epoch 3/20
10/10 [==============================] - 0s 5ms/step - loss: 2.8879 - accuracy: 0.3000
Epoch 4/20
10/10 [==============================] - 0s 5ms/step - loss: 2.2674 - accuracy: 0.3000
Epoch 5/20
10/10 [==============================] - 0s 5ms/step - loss: 2.0133 - accuracy: 0.2000
Epoch 6/20
10/10 [==============================] - 0s 5ms/step - loss: 1.8272 - accuracy: 0.3000
Epoch 7/20
10/10 [==============================] - 0s 5ms/step - loss: 1.7794 - accuracy: 0.4000
Epoch 8/20
10/10 [==============================] - 0s 6ms/step - loss: 1.8607 - accuracy: 0.3000
Epoch 9/20
10/10 [==============================] - 0s 7ms/step - loss: 1.8102 - accuracy: 0.2000
Epoch 10/20
10/10 [==============================] - 0s 6ms/step - loss: 1.8010 - accuracy: 0.1000
Epoch 11/20
10/10 

If it does not overfit try to fix data prep and model architecture so it does

It overfit

Create a function **make_prediction(seed_start)** which takes a starting string sequence and uses it to generate a molecule


In [ ]:
# Create a function make_prediction(seed_start) wich takes a starting sequence and uses it to generate a molecule

def make_prediction(seed_start):
    seed = [dict_char_to_int[i] for i in seed_start]
    seed = np.array(seed)
    seed = seed.reshape(1, seq_length, 1)
    seed = seed / len(unic_chars)
    
    prediction = model.predict(seed)
    
    prediction = np.argmax(prediction)
    
    return seed_start + dict_int_to_char[prediction]

generate a molecule of your overfitted model

In [ ]:
# Test it with a random seed
print(make_prediction("CCOCOCC"))

Make a model checkpoint so that the model is saved after each epoch
if you train on a plateform and it stops you do not lose your training 

Now go to your favorite plateform (colab or something else) and train the dataset on the whole data for 10 epochs and batch size 256 

it should take a long time so either follow the class or go take a nap 

Generate between 100 and 1000 molecules. 

create a list where molecules have between 10 and 50 atoms

With rdkit compute the Quantified Estimated Drug likelyness (QED) of each molecule in this subset

Bonus 1 : Using rdkit, compute the quantitative estimation of drug-likeness (QED) of your generated molecules. 

Bonus 2 : try to adapt a transformer model training from hugging face to see if it is better